In [38]:
import json
import requests
import time as t
import pandas as pd
import matplotlib.pyplot as plt
from citipy import citipy as c
import random as r

date = t.strftime("%d %b %Y", t.gmtime())



In [39]:
x = t.time()
print(x)
while t.time() < (x + 1):
    t.sleep(.1)
print(t.time())

1555538351.9874437
1555538352.9947507


In [54]:
for i in range(20):
    lat = r.uniform(-1, 1) * 90
    lng = r.uniform(-1, 1) * 180
    a = c.nearest_city(lat, lng)
    print(lat, lng)
    print(a.city_name, a.country_code)

25.502257429768306 123.68093978151566
ishigaki jp
81.50539370031834 -161.4688926481841
barrow us
56.82842497733935 59.036487381412535
bisert ru
59.86497733460496 -149.29835357875993
anchorage us
-7.71467242343435 -104.48887446168773
puerto ayora ec
47.69373062361269 142.50300790407803
vzmorye ru
-12.360565377027173 8.507292762758052
namibe ao
50.10201811679755 8.838450954812277
muhlheim de
-17.062902609531502 -155.34843665463623
vaitape pf
24.63093508093239 -143.71191082603494
hilo us
65.48651109450579 104.02542162241262
tura ru
65.27798655226333 -153.0572374636855
kenai us
63.58823948640949 -40.86662045259476
tasiilaq gl
-69.55267774895333 144.9069639954504
hobart au
82.71677631015321 -60.14965439147449
narsaq gl
-58.74944731573062 -31.697863181789625
mar del plata ar
66.62060737291803 162.5662366047508
cherskiy ru
39.59427008515032 -133.83016293991753
fortuna us
89.19002803222523 163.44012809571805
cherskiy ru
73.50018711949149 137.76047207692037
nizhneyansk ru
